# GW170817 Downloading and processing

In [ ]:
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.frame import read_frame
from pycbc.filter import highpass, resample_to_delta_t
import urllib.request

m = Merger("GW170817")

# List of observatories we'll analyze
ifos = ['H1', 'V1', 'L1']

# The single template waveform model needs these data products
psds = {}
data = {}

for ifo in ifos:
    print(f"Processing {ifo} data")
    
    # Construct the URL and local filename
    url_template = "https://dcc.ligo.org/public/0146/P1700349/001/{}-{}1_LOSC_CLN_4_V1-1187007040-2048.gwf"
    formatted_url = url_template.format(ifo[0], ifo[0])
    filename = formatted_url.split('/')[-1]  # Extracts the filename from the URL
    
    # Download the file to the current directory
    urllib.request.urlretrieve(formatted_url, filename)
    fname = filename  # Use the local file path
    
    # Read and condition the data
    ts = read_frame(fname, f"{ifo}:LOSC-STRAIN",
                    start_time=int(m.time - 260),
                    end_time=int(m.time + 40))
    ts = highpass(ts, 15.0)
    ts = resample_to_delta_t(ts, 1.0/2048)
    ts = ts.time_slice(m.time - 112, m.time + 16)
    data[ifo] = ts.to_frequencyseries()

    # Estimate the PSD
    psd = interpolate(ts.psd(4), ts.delta_f)
    psd = inverse_spectrum_truncation(psd, int(4 * psd.sample_rate), 
                                      trunc_method='hann',
                                      low_frequency_cutoff=20.0)
    psds[ifo] = psd